### Data Preprocessing

In [1]:
!ls /content/

'archive (4).zip'   sample_data   scraped.zip


In [2]:
import os

os.listdir("/content")


['.config',
 '.ipynb_checkpoints',
 'archive (4).zip',
 'scraped.zip',
 'sample_data']

In [3]:
!file scraped.zip


scraped.zip: Zip archive data, at least v4.5 to extract, compression method=deflate


In [4]:
!apt-get update
!apt-get install -y p7zip-full


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,205 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy InRelease

In [5]:
!7z x scraped.zip -o/content/dataset_raw



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 106954752 bytes (102 MiB)

Extracting archive: scraped.zip

ERRORS:
Unexpected end of archive

--
Path = scraped.zip
Type = zip
ERRORS:
Unexpected end of archive
Physical Size = 261791118

  0%      3% - scraped_data.txt                         5% - scraped_data.txt                         7% - scraped_data.txt                        10% - scraped_data.txt                        12% - scraped_data.txt                        14% - scraped_data.txt              

In [6]:
import os

os.listdir("/content/dataset_raw")


['scraped_data.txt']

In [7]:
file_path = "/content/dataset_raw/scraped_data.txt"  # adjust name

with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    for i in range(5):
        print(f.readline())


Title: Wikipedia, the free encyclopedia



Content: Wikipedia, the free encyclopedia Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Donate Contribute Help Learn to edit Community portal Recent changes Upload file Search Search Create account Log in Personal tools Create account Log in Pages for logged out editors learn more Contributions Talk Main Page Main Page Talk English Read View source View history Tools Tools move to sidebar hide Actions Read View source View history General What links here Related changes Upload file Special pages Permanent link Page information Cite this page Get shortened URL Wikidata item Print/export Download as PDF Printable version In other projects Wikimedia Commons Wikimedia Foundation MediaWiki Meta-Wiki Wikimedia Outreach Multilingual Wikisource Wikispecies Wikibooks Wikidata Wikifunctions Wikimania Wikinews Wikiquote Wikisource Wikiversity Wikivoyage Wikt

In [8]:
import os, shutil

base_path = "/content/text_generation_rnn"
os.makedirs(base_path + "/dataset/raw", exist_ok=True)

shutil.move(
    "/content/dataset_raw/scraped_data.txt",
    base_path + "/dataset/raw/wiki_corpus.txt"
)

print("Dataset ready ✔")


Dataset ready ✔


## Tokenizing

In [1]:
import re
import itertools
from tensorflow.keras.preprocessing.text import Tokenizer

RAW_TEXT_PATH = "/content/text_generation_rnn/dataset/raw/wiki_corpus.txt"

MAX_LINES = 900_000   # SAFE LIMIT (~5–10MB text)

lines = []

with open(RAW_TEXT_PATH, "r", encoding="utf-8", errors="ignore") as f:
    for line in itertools.islice(f, MAX_LINES):
        lines.append(line)

print("Lines loaded:", len(lines))

text = " ".join(lines)

# Clean
text = text.lower()
text = re.sub(r"[^a-z\s]", "", text)
text = re.sub(r"\s+", " ", text)

print("Text length:", len(text))

# Tokenizer
tokenizer = Tokenizer(num_words=20000, oov_token="<unk>")
tokenizer.fit_on_texts([text])

print("Vocabulary size:", min(len(tokenizer.word_index), 20000))


Lines loaded: 312932
Text length: 287222746
Vocabulary size: 20000


In [2]:
import numpy as np

SEQ_LEN = 10
MAX_SAMPLES = 300_000   # HARD LIMIT (do not increase now)

# Convert text → word indices
tokens = tokenizer.texts_to_sequences([text])[0]
print("Total tokens:", len(tokens))

X = []
y = []

for i in range(SEQ_LEN, min(len(tokens), MAX_SAMPLES)):
    X.append(tokens[i-SEQ_LEN:i])
    y.append(tokens[i])

X = np.array(X, dtype=np.int32)
y = np.array(y, dtype=np.int32)

print("X shape:", X.shape)
print("y shape:", y.shape)


Total tokens: 45494891
X shape: (299990, 10)
y shape: (299990,)


In [3]:
import json
import os

PROCESSED_PATH = "/content/text_generation_rnn/dataset/processed"
os.makedirs(PROCESSED_PATH, exist_ok=True)

np.save(PROCESSED_PATH + "/input_sequences.npy", X)
np.save(PROCESSED_PATH + "/output_words.npy", y)

with open(PROCESSED_PATH + "/word2idx.json", "w") as f:
    json.dump(tokenizer.word_index, f)

print("✅ Dataset saved successfully")


✅ Dataset saved successfully


### Buildig model: LSTM

In [4]:
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths
DATA_PATH = "/content/text_generation_rnn/dataset/processed"
MODEL_PATH = "/content/text_generation_rnn/models/lstm_text_generator.h5"

# Load dataset
X = np.load(DATA_PATH + "/input_sequences.npy")
y = np.load(DATA_PATH + "/output_words.npy")

with open(DATA_PATH + "/word2idx.json", "r") as f:
    word2idx = json.load(f)

vocab_size = min(len(word2idx) + 1, 20000)
seq_len = X.shape[1]

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Vocab size:", vocab_size)

# -----------------------------
# Build Advanced LSTM Model
# -----------------------------
model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=128,
        input_length=seq_len
    ),

    LSTM(256, return_sequences=True),
    Dropout(0.3),

    LSTM(256),
    Dropout(0.3),

    Dense(256, activation="relu"),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]  # accuracy is logged but NOT trusted
)

model.summary()


X shape: (299990, 10)
y shape: (299990,)
Vocab size: 20000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        MODEL_PATH,
        monitor="val_loss",
        save_best_only=True
    )
]

history = model.fit(
    X,
    y,
    batch_size=128,
    epochs=20,
    validation_split=0.1,
    callbacks=callbacks
)


Epoch 1/20
2109/2110 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1050 - loss: 6.9302

2110/2110 ━━━━━━━━━━━━━━━━━━━━ 48s 18ms/step - accuracy: 0.1051 - loss: 6.9298 - val_accuracy: 0.2331 - val_loss: 6.8758
Epoch 2/20
2107/2110 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1894 - loss: 5.6865

2110/2110 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.1894 - loss: 5.6863 - val_accuracy: 0.2603 - val_loss: 6.7860
Epoch 3/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.2340 - loss: 5.1714 - val_accuracy: 0.2651 - val_loss: 6.9494
Epoch 4/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.2561 - loss: 4.8293 - val_accuracy: 0.2696 - val_loss: 7.1402
Epoch 5/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.2736 - loss: 4.5588 - val_accuracy: 0.2662 - val_loss: 7.3859


## Testing GRU model

In [11]:
print(generate_text("deep learning is a field that", temperature=0.7, top_k=10))


deep learning is a field that and to the


In [12]:
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
DATA_PATH = "/content/text_generation_rnn/dataset/processed"

X = np.load(DATA_PATH + "/input_sequences.npy")
y = np.load(DATA_PATH + "/output_words.npy")

with open(DATA_PATH + "/word2idx.json", "r") as f:
    word2idx = json.load(f)

vocab_size = 20000
seq_len = X.shape[1]

print("X:", X.shape)
print("y:", y.shape)

# ---------------------------
# GRU MODEL (BEST FOR YOU)
# ---------------------------
model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=128,
        input_shape=(seq_len,)
    ),

    GRU(256, return_sequences=True),
    Dropout(0.3),

    GRU(256),
    Dropout(0.3),

    Dense(256, activation="relu"),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


X: (299990, 10)
y: (299990,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 10, 128)        │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 10, 256)        │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 256)            │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20000)          │     5,140,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,456,992 (32.26 MB)

 Trainable params: 8,456,992 (32.26 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(
    X,
    y,
    batch_size=128,
    epochs=8,
    validation_split=0.1,
    callbacks=[
        EarlyStopping(
            monitor="val_loss",
            patience=2,
            restore_best_weights=True
        )
    ]
)


Epoch 1/8
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.1145 - loss: 6.8457 - val_accuracy: 0.2507 - val_loss: 6.8125
Epoch 2/8
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 37s 18ms/step - accuracy: 0.2120 - loss: 5.5506 - val_accuracy: 0.2602 - val_loss: 6.9089
Epoch 3/8
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 37s 18ms/step - accuracy: 0.2476 - loss: 5.0988 - val_accuracy: 0.2637 - val_loss: 7.0430


In [17]:
def generate_text(
    seed_text,
    num_words=30,
    temperature=0.7,
    top_k=10,
    repetition_penalty=1.2
):
    result = seed_text.lower().split()

    for _ in range(num_words):
        encoded = [word2idx.get(w, word2idx["<unk>"]) for w in result[-SEQ_LEN:]]
        padded = pad_sequences([encoded], maxlen=SEQ_LEN)

        preds = model.predict(padded, verbose=0)[0]

        # 🔥 repetition penalty
        for w in set(result[-5:]):
            if w in word2idx:
                preds[word2idx[w]] /= repetition_penalty

        preds = np.log(preds + 1e-8) / temperature
        probs = np.exp(preds) / np.sum(np.exp(preds))

        top_indices = np.argsort(probs)[-top_k:]
        top_probs = probs[top_indices]
        top_probs /= np.sum(top_probs)

        next_idx = np.random.choice(top_indices, p=top_probs)
        next_word = idx2word.get(next_idx, "")

        if next_word in ["<unk>", result[-1]]:
            continue

        result.append(next_word)

    return " ".join(result)


In [19]:
print(
    generate_text(
        "deep learning is a field of artificial intelligence that",
        temperature=0.6,
        top_k=8,
        repetition_penalty=1.3
    )
)


deep learning is a field of artificial intelligence that a nobel prize is to a university of the national university of the prize of the university of


### Trying to build transformer - mini gpt

In [21]:
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras import layers, models

# =========================
# LOAD DATA
# =========================
DATA_PATH = "/content/text_generation_rnn/dataset/processed"

X = np.load(DATA_PATH + "/input_sequences.npy")
y = np.load(DATA_PATH + "/output_words.npy")

with open(DATA_PATH + "/word2idx.json", "r") as f:
    word2idx = json.load(f)

idx2word = {v: k for k, v in word2idx.items()}

VOCAB_SIZE = 20000
SEQ_LEN = X.shape[1]

print("X:", X.shape, "y:", y.shape)

# =========================
# TRANSFORMER BLOCK
# =========================
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(
            inputs, inputs,
            use_causal_mask=True
        )
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)

        return self.layernorm2(out1 + ffn_output)

# =========================
# TOKEN + POSITION EMBEDDING
# =========================
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(vocab_size, embed_dim)
        self.pos_emb = layers.Embedding(maxlen, embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# =========================
# MINI GPT MODEL
# =========================
EMBED_DIM = 128
NUM_HEADS = 4
FF_DIM = 256

inputs = layers.Input(shape=(SEQ_LEN,))
embedding_layer = TokenAndPositionEmbedding(SEQ_LEN, VOCAB_SIZE, EMBED_DIM)
x = embedding_layer(inputs)

x = TransformerBlock(EMBED_DIM, NUM_HEADS, FF_DIM)(x)
x = TransformerBlock(EMBED_DIM, NUM_HEADS, FF_DIM)(x)

x = layers.Dense(EMBED_DIM, activation="relu")(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x[:, -1, :])

model = models.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


X: (299990, 10) y: (299990,)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_1  │ (None, 10, 128)        │     2,561,280 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ (None, 10, 128)        │       330,240 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ (None, 10, 128)        │       330,240 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10, 128)        │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ get_item (GetItem)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 20000)          │     2,580,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,818,272 (22.19 MB)

 Trainable params: 5,818,272 (22.19 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(
    X,
    y,
    batch_size=128,
    epochs=5,
    validation_split=0.1
)


Epoch 1/5
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - accuracy: 0.1372 - loss: 6.7496 - val_accuracy: 0.2536 - val_loss: 6.7952
Epoch 2/5
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.2560 - loss: 5.1418 - val_accuracy: 0.2750 - val_loss: 6.8031
Epoch 3/5
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.3009 - loss: 4.5089 - val_accuracy: 0.2714 - val_loss: 7.1445
Epoch 4/5
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.3348 - loss: 4.0212 - val_accuracy: 0.2647 - val_loss: 7.7825
Epoch 5/5
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.3651 - loss: 3.6163 - val_accuracy: 0.2653 - val_loss: 8.4094


## Final testing

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_text_transformer(
    seed_text,
    num_words=30,
    temperature=0.8,
    top_k=10
):
    result = seed_text.lower().split()

    for _ in range(num_words):
        encoded = [word2idx.get(w, word2idx["<unk>"]) for w in result[-SEQ_LEN:]]
        padded = pad_sequences([encoded], maxlen=SEQ_LEN)

        preds = model.predict(padded, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        probs = np.exp(preds) / np.sum(np.exp(preds))

        top_indices = np.argsort(probs)[-top_k:]
        top_probs = probs[top_indices]
        top_probs /= np.sum(top_probs)

        next_idx = np.random.choice(top_indices, p=top_probs)
        next_word = idx2word.get(next_idx, "")

        if next_word == "<unk>":
            continue

        result.append(next_word)

    return " ".join(result)

print(
    generate_text_transformer(
        "deep learning is a field of artificial intelligence that",
        temperature=0.7,
        top_k=10
    )
)


deep learning is a field of artificial intelligence that are also used to have been blocked for other editors and the most common encyclopedia in english is an official language for the encyclopedia of the united nations
